<a href="https://colab.research.google.com/github/SaumyaGupta-99/sarcasm_detection/blob/master/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import seaborn as sns
import nltk
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# import json
# with open('/content/drive/My Drive/sarcasm_detection/Sarcasm_Headlines_Dataset_v2.json') as f:
#    data1 = json.load(f)
# print (data1)
# with open('/content/drive/My Drive/sarcasm_detection/Sarcasm_Headlines_Dataset.json') as f:
#    data2 = json.load(f)
# print (data2)
train1=pd.read_json('/content/drive/My Drive/sarcasm_detection/Sarcasm_Headlines_Dataset_v2.json',lines=True)
train2=pd.read_json("/content/drive/My Drive/sarcasm_detection/Sarcasm_Headlines_Dataset.json",lines=True)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
train1.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
data=train1.append(train2,ignore_index=True)
dataorg=train1.append(train2,ignore_index=True)
data.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [ ]:
data['headline']=data['headline'].str.replace("[^a-zA-Z]",' ')
data['headline'].head()
hl_cleansed = []

In [ ]:
data['headline']=data['headline'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
data['headline'].head()

0    thirtysomething scientists unveil doomsday clo...
1    dem rep totally nails why congress falling sho...
2       eat your veggies deliciously different recipes
3    inclement weather prevents liar from getting work
4    mother comes pretty close using word streaming...
Name: headline, dtype: object

In [ ]:
tokenized_words=data['headline'].apply(lambda x: x.split())
tokenized_words.head()

0    [thirtysomething, scientists, unveil, doomsday...
1    [dem, rep, totally, nails, why, congress, fall...
2    [eat, your, veggies, deliciously, different, r...
3    [inclement, weather, prevents, liar, from, get...
4    [mother, comes, pretty, close, using, word, st...
Name: headline, dtype: object

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()   
tokenized_words=tokenized_words.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
tokenized_words.head()

0    [thirtysomething, scientist, unveil, doomsday,...
1    [dem, rep, totally, nail, why, congress, falli...
2    [eat, your, veggie, deliciously, different, re...
3    [inclement, weather, prevents, liar, from, get...
4    [mother, come, pretty, close, using, word, str...
Name: headline, dtype: object

In [ ]:
for i in range(len(tokenized_words)):
  tokenized_words[i]=" ".join(tokenized_words[i])
data['headline']=tokenized_words
data['headline'].head()

0    thirtysomething scientist unveil doomsday cloc...
1    dem rep totally nail why congress falling shor...
2         eat your veggie deliciously different recipe
3    inclement weather prevents liar from getting work
4    mother come pretty close using word streaming ...
Name: headline, dtype: object

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer=CountVectorizer(max_df=0.80,min_df=2,stop_words='english')
bow=bow_vectorizer.fit_transform(data['headline'])

In [ ]:
n=(int)(len(data)*0.9)
print(n)

49795


In [ ]:
#Decisiontree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_bow = bow[:n]
test_bow = bow[n:]
y=data['is_sarcastic'][:n]
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow,y,random_state=42, test_size=0.3)
clf = DecisionTreeClassifier()
clf = clf.fit(xtrain_bow,ytrain)
prediction=clf.predict_proba(xvalid_bow)
prediction_int = prediction[:,1] >= 0.3
prediction_int=prediction_int.astype(np.int)
f1_score(yvalid, prediction_int)

0.8710440698018337

In [ ]:
test_pred = clf.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test=data[n:]
test['is_sarcastic'] = test_pred_int
test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,is_sarcastic,headline,article_link
49795,0,how transform your relationship with money pla...,https://www.huffingtonpost.com/entry/how-to-tr...
49796,1,george kennedy honor riding internal breath fr...,https://www.theonion.com/george-kennedys-honor...
49797,1,woman strike sobering reality check for subway...,https://local.theonion.com/women-s-strike-a-so...
49798,1,video game bos thinking should get big glowing...,https://www.theonion.com/video-game-boss-think...
49799,1,man worried any crazy person could get hand co...,https://local.theonion.com/man-worried-any-cra...


In [ ]:
res=clf.predict(test_bow[:10])
Y_test=dataorg['headline'][n:]
d = {'Headlines':Y_test[:10],'Result':res}
pd.DataFrame(d)

,Headlines,Result
49795,how to transform your relationship with money ...,0
49796,george kennedy's honor riding on internal brea...,1
49797,women's strike a sobering reality check for su...,1
49798,video game boss thinking he should get big glo...,1
49799,man worried any crazy person could get hands o...,1
49800,romney tells heartbreaking lie about single mo...,1
49801,life-changing epiphany wears off on ride home,1
49802,are there tears in your popcorn? what to learn...,0
49803,"he told his boyfriend, 'i love you.' his boyfr...",0
49804,an art project over 40 years in the making let...,0


In [ ]:
#SVM
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_bow = bow[:n]
test_bow = bow[n:]
y=data['is_sarcastic'][:n]
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow,y,random_state=42, test_size=0.3)
lsvc = LinearSVC()
lsvc = lsvc.fit(xtrain_bow,ytrain)
# prediction=lsvc.predict_proba(xvalid_bow)
# prediction_int = prediction[:,1] >= 0.3
# prediction_int=prediction_int.astype(np.int)
f1_score(yvalid, lsvc.predict(xvalid_bow))

0.877662721893491

In [ ]:
res=lsvc.predict(test_bow[:10])
testhead=dataorg['headline'][n:]
d = {'Headlines':testhead[:10],'Result':res}
pd.DataFrame(d)

,Headlines,Result
49795,how to transform your relationship with money ...,0
49796,george kennedy's honor riding on internal brea...,1
49797,women's strike a sobering reality check for su...,1
49798,video game boss thinking he should get big glo...,1
49799,man worried any crazy person could get hands o...,1
49800,romney tells heartbreaking lie about single mo...,1
49801,life-changing epiphany wears off on ride home,0
49802,are there tears in your popcorn? what to learn...,0
49803,"he told his boyfriend, 'i love you.' his boyfr...",1
49804,an art project over 40 years in the making let...,0


In [ ]:
#Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_bow = bow[:n]
test_bow = bow[n:]
y=data['is_sarcastic'][:n]
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow,y,random_state=42, test_size=0.3)
clfrf=RandomForestClassifier(n_estimators=100)
clfrf = clfrf.fit(xtrain_bow,ytrain)
prediction=clfrf.predict_proba(xvalid_bow)
prediction_int = prediction[:,1] >= 0.3
prediction_int=prediction_int.astype(np.int)
f1_score(yvalid, prediction_int)

0.8811111111111111

In [ ]:
test_pred = clfrf.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.5
test_pred_int = test_pred_int.astype(np.int)
test=data[n:]
test['is_sarcastic'] = test_pred_int

In [ ]:
res=clfrf.predict(test_bow[:10])
Y_test=dataorg['headline'][n:]
d = {'Headlines':Y_test[:10],'Result':res}
pd.DataFrame(d)

,Headlines,Result
49795,how to transform your relationship with money ...,0
49796,george kennedy's honor riding on internal brea...,1
49797,women's strike a sobering reality check for su...,1
49798,video game boss thinking he should get big glo...,0
49799,man worried any crazy person could get hands o...,1
49800,romney tells heartbreaking lie about single mo...,1
49801,life-changing epiphany wears off on ride home,1
49802,are there tears in your popcorn? what to learn...,0
49803,"he told his boyfriend, 'i love you.' his boyfr...",0
49804,an art project over 40 years in the making let...,0


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np
max_features=2000
max_token = len(max(data['headline']))
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['headline'])
sequences = tokenizer.texts_to_sequences(data['headline'])
X = pad_sequences(sequences, maxlen=max_token)
from sklearn.model_selection import train_test_split
Y = data['is_sarcastic'].values
Y = np.vstack(Y)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3, random_state = 42)
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
embed_dim = 64
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length =(max_token)))
model.add(LSTM(96, dropout=0.2, recurrent_dropout=0.2, activation='relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 96, 64)            128000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 96)                61824     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 97        
_________________________________________________________________
activation_4 (Activation)    (None, 1)                 0         
Total params: 189,921
Trainable params: 189,921
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
train_bow.shape

In [ ]:
epoch = 20
batch_size = 128
model.fit(X_train, Y_train, epochs = epoch, batch_size=batch_size, verbose = 2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 - 82s - loss: 0.5590 - accuracy: 0.7649
Epoch 2/20
 - 82s - loss: 0.3855 - accuracy: 0.8553
Epoch 3/20
 - 82s - loss: 0.3116 - accuracy: 0.8673
Epoch 4/20
 - 82s - loss: 0.2950 - accuracy: 0.8730
Epoch 5/20
 - 82s - loss: 53783.3044 - accuracy: 0.8730
Epoch 6/20
 - 81s - loss: 1450.1442 - accuracy: 0.8711
Epoch 7/20
 - 81s - loss: 0.2851 - accuracy: 0.8783
Epoch 8/20
 - 81s - loss: 0.2759 - accuracy: 0.8805
Epoch 9/20
 - 82s - loss: 0.2678 - accuracy: 0.8844
Epoch 10/20
 - 82s - loss: 0.2872 - accuracy: 0.8845
Epoch 11/20
 - 81s - loss: 0.2589 - accuracy: 0.8884
Epoch 12/20
 - 82s - loss: 0.2549 - accuracy: 0.8883
Epoch 13/20
 - 82s - loss: 0.2497 - accuracy: 0.8912
Epoch 14/20
 - 82s - loss: 0.2465 - accuracy: 0.8935
Epoch 15/20
 - 82s - loss: 0.2402 - accuracy: 0.8948
Epoch 16/20
 - 82s - loss: 0.2330 - accuracy: 0.8980
Epoch 17/20
 - 81s - loss: 0.2277 - accuracy: 0.9017
Epoch 18/20
 - 82s - loss: 0.2220 - accuracy: 0.9043
Epoch 19/20
 - 81s - loss: 0.2166 - accuracy: 0.

In [ ]:
result = model.predict(X_test)
crr=0
for i in range(len(X_test)):
  if(np.around(result[i])==np.around(Y_test[i])):
    crr=crr+1
acc=crr/(len(X_test))
print(acc)

0.8799927706488343


In [ ]:
res=model.predict(X_test[:10])
res2=[]
for i in range(10):
  res2.append(int(np.around(res[i])))
testhead=dataorg['headline'][n:]
d = {'Headlines':testhead[:10],'Result':res2}
pd.DataFrame(d)

,Headlines,Result
49795,how to transform your relationship with money ...,0
49796,george kennedy's honor riding on internal brea...,0
49797,women's strike a sobering reality check for su...,0
49798,video game boss thinking he should get big glo...,1
49799,man worried any crazy person could get hands o...,0
49800,romney tells heartbreaking lie about single mo...,0
49801,life-changing epiphany wears off on ride home,1
49802,are there tears in your popcorn? what to learn...,1
49803,"he told his boyfriend, 'i love you.' his boyfr...",0
49804,an art project over 40 years in the making let...,0


In [ ]:
resy=[]
for i in range(10):
  resy.append(int(np.around(Y_test[i])))
d = {'Headlines':X_test[:10],'Result':resy}
pd.DataFrame(d)
#print(Y_test[:10])

,Headlines,Result
49795,how to transform your relationship with money ...,0
49796,george kennedy's honor riding on internal brea...,0
49797,women's strike a sobering reality check for su...,0
49798,video game boss thinking he should get big glo...,1
49799,man worried any crazy person could get hands o...,0
49800,romney tells heartbreaking lie about single mo...,0
49801,life-changing epiphany wears off on ride home,0
49802,are there tears in your popcorn? what to learn...,1
49803,"he told his boyfriend, 'i love you.' his boyfr...",0
49804,an art project over 40 years in the making let...,0
